In [ ]:
#part1
import random

class SimpleTradeStrategy:
    def __init__(self, loss_limit, profit_target, position_size):
        self.loss_limit = loss_limit
        self.profit_target = profit_target
        self.position_size = position_size
    def encode(self):
        return f"{int(self.loss_limit * 10):02}{int(self.profit_target):02}{int(self.position_size):02}"

    @staticmethod
    def decode(strategy_string):
        loss_limit = int(strategy_string[:2]) / 10
        profit_target = int(strategy_string[2:4])
        position_size = int(strategy_string[4:])
        return SimpleTradeStrategy(loss_limit, profit_target, position_size)

    def __repr__(self):
        return f"{{'Loss Limit': {self.loss_limit}, 'Profit Target': {self.profit_target}, 'Position Size': {self.position_size}}}"
def create_population():
    return [
        SimpleTradeStrategy(2, 5, 20),
        SimpleTradeStrategy(3, 7, 30),
        SimpleTradeStrategy(1.5, 4, 25),
        SimpleTradeStrategy(2.5, 6, 15)
    ]

def evaluate_strategy(strategy, market_data, starting_balance=1000):
    balance = starting_balance
    for price_change in market_data:
        trade_amount = (strategy.position_size / 100) * balance
        if price_change <= -strategy.loss_limit:
            loss = trade_amount * (strategy.loss_limit / 100)
            balance -= loss
        elif price_change >= strategy.profit_target:
            profit = trade_amount * (strategy.profit_target / 100)
            balance += profit
        else:
            result = trade_amount * (price_change / 100)
            balance += result
    return balance - starting_balance

def select_parents(population, market_data):
    return random.sample(population, 2)

def crossover(parent1, parent2):
    point = random.randint(1, 5)
    child1 = parent1.encode()[:point] + parent2.encode()[point:]
    child2 = parent2.encode()[:point] + parent1.encode()[point:]
    return SimpleTradeStrategy.decode(child1), SimpleTradeStrategy.decode(child2)

def mutate(strategy, mutation_rate=0.05):
    if random.random() < mutation_rate:
        parameter = random.choice(["loss_limit", "profit_target", "position_size"])
        if parameter == "loss_limit":
            strategy.loss_limit = round(random.uniform(1, 99), 1)
        elif parameter == "profit_target":
            strategy.profit_target = random.randint(1, 99)
        else:
            strategy.position_size = random.randint(1, 99)
    return strategy

def evolve_population(population, market_data):
    sorted_population = sorted(population, key=lambda s: evaluate_strategy(s, market_data), reverse=True)[:2]

    while len(sorted_population) < len(population):
        parent1, parent2 = select_parents(population, market_data)
        child1, child2 = crossover(parent1, parent2)
        sorted_population.extend([mutate(child1), mutate(child2)])

    return sorted_population[:len(population)]

def run_genetic_algorithm(num_generations=10):
    market_data = [-1.2, 3.4, -0.8, 2.1, -2.5, 1.7, -0.3, 5.8, -1.1, 3.5]
    population = create_population()

    best_strategy = None
    best_profit = float('-inf')

    for generation in range(num_generations):
        population = evolve_population(population, market_data)
        top_strategy = max(population, key=lambda s: evaluate_strategy(s, market_data))
        top_profit = evaluate_strategy(top_strategy, market_data)

        if top_profit > best_profit:
            best_strategy = top_strategy
            best_profit = top_profit

    return best_strategy, best_profit

best_strategy, profit = run_genetic_algorithm(10)
print({
    "Best Strategy": {
        "stop_loss": best_strategy.loss_limit,
        "take_profit": best_strategy.profit_target,
       "trade_size": best_strategy.position_size
    },
    "Final Profit": profit
})

#part2
import random


def crossover(parent1, parent2):
    parent1_chromosome = parent1.encode()
    parent2_chromosome = parent2.encode()

    point1 = random.randint(1, len(parent1_chromosome) - 2)
    point2 = random.randint(point1 + 1, len(parent1_chromosome) - 1)

    offspring1_chromosome = parent1_chromosome[:point1] + parent2_chromosome[point1:point2] + parent1_chromosome[point2:]
    offspring2_chromosome = parent2_chromosome[:point1] + parent1_chromosome[point1:point2] + parent2_chromosome[point2:]

    return offspring1_chromosome, offspring2_chromosome

parent1 = SimpleTradeStrategy(2, 5, 20)
parent2 = SimpleTradeStrategy(3, 7, 30)

offspring1, offspring2 = crossover(parent1, parent2)

print("The two resultant offsprings are, {} & {}".format(offspring1, offspring2))




{'Best Strategy': {'stop_loss': 1.0, 'take_profit': 7, 'trade_size': 53}, 'Final Profit': 66.91596153540104}
